In [1]:
import numpy as np
import pandas as pd
from model import BatchProgramCC
from gensim.models.word2vec import Word2Vec
import torch

def get_device():
    if torch.cuda.is_available():
        if torch.cuda.get_device_name(0) == 'GeForce GT 730':
            device = 'cpu'
        else:
            device = 'cuda'
    else:
        device = 'cpu'
    return torch.device(device)

device = get_device()


root = 'data/'

train_data = pd.read_pickle(root+'train/blocks.pkl').sample(frac=1.0)
validation_data = pd.read_pickle(root + 'dev/blocks.pkl').sample(frac=1.0)
test_data = pd.read_pickle(root+'test/blocks.pkl').sample(frac=1.0)
ast_embedded = pd.concat([train_data, validation_data, test_data], ignore_index=True)
del train_data
del validation_data
del test_data

#ast_embedded = pd.read_pickle(root+lang+'/ast_embedded.pkl')

word2vec = Word2Vec.load(root+"train/embedding/node_w2v_128").wv
MAX_TOKENS = word2vec.vectors.shape[0]
EMBEDDING_DIM = word2vec.vectors.shape[1]
#embeddings = np.zeros((MAX_TOKENS + 1, EMBEDDING_DIM), dtype="float32")
#embeddings[:MAX_TOKENS] = word2vec.vectors

C:\Users\A1\anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:102: UserWarning: 
    Found GPU0 GeForce GT 730 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))
C:\Users\A1\anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:125: UserWarning: 
GeForce GT 730 with CUDA capability sm_30 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the GeForce GT 730 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [2]:
import math

class dataloader():
    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.idx = 0
        self.max_idx = len(dataset)
        self.len = math.ceil(len(self.dataset) / self.batch_size)
        
    def __call__(self):
        pass
    
    def __len__(self):
        return self.len
    
    def __iter__(self):
        self.idx = 0
        return self
    
    def __next__(self):
        if self.idx >= self.max_idx:
            raise StopIteration
        tmp = self.dataset.iloc[self.idx: self.idx+self.batch_size]
        x = []
        for _, item in tmp.iterrows():
            x.append(item['code'])
        self.idx += self.batch_size
        return x
    


In [3]:
HIDDEN_DIM = 100
ENCODE_DIM = 128
LABELS = 104
BATCH_SIZE = 128

model_file = '20201006_151536 model for classification task/20201007_013732 model for classification task epoch_35 val.pt'
save_dir = '---- classification model/'


model = BatchProgramCC(EMBEDDING_DIM,
                       HIDDEN_DIM,
                       MAX_TOKENS+1,
                       ENCODE_DIM,
                       LABELS,
                       BATCH_SIZE,
                       device).to(device)

data = dataloader(ast_embedded, BATCH_SIZE)

In [4]:
param = torch.load(save_dir + model_file)['param']
model.load_state_dict(param)
model.eval()

BatchProgramCC(
  (encoder): BatchTreeEncoder(
    (embedding): Embedding(8189, 128)
    (W_c): Linear(in_features=128, out_features=128, bias=True)
  )
  (bigru): GRU(128, 100, batch_first=True, bidirectional=True)
  (hidden2label): Linear(in_features=200, out_features=104, bias=True)
)

In [6]:
with open(root+'Xmeans dataset/cluster data.txt', 'w') as f:
    with torch.no_grad():
        for x in data:
            model.batch_size = len(x)
            model.hidden = model.init_hidden()
            encode = model.encode(x).cpu().numpy()
            buffer = []
            for s in encode:
                i = [str(i) for i in s]
                i = ', '.join(i)
                buffer.append(i)
            buffer = '\n'.join(buffer)
            f.write(buffer+'\n')
            #break